<a href="https://colab.research.google.com/github/NahuelRepetto/Programacion-Concurrente/blob/main/TP2_P2_OpenMPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programacion Concurrente TP2 Parte2 MPI 

Para este ejercicio se ha optado por aplicar el tema teórico **MPI** (Message Passing Interface). La finalidad del ejercicio es ampliar el conocimiento sobras la manera que posee Python para implementar la comunicación entre distintos procesos con el uso de una librería denominada **MPI4py** [1]. Para más información puede consultar la última revisión en [2].

Este ejercicio fue presentado por alumnos del curso 2020.

---
## 2.1. Ejercicio - Hola Mundo con MPI


### 2.1.1. Preguntas del ejercicio

Ejecute este ejemplo para $4$ o más instancias y responda:

a) ¿Qué función realiza la instancia maestra? ¿Qué función realizan las instancias esclavas?

b) ¿Cuántas de esas instancias ejecuta la función `main()`, `initWork()` y `doWork()`?

c) ¿Cómo se diferencian los mensajes de trabajo o de finalización?

d) ¿Cómo implementaría la función BLAS `axpy()` con este patrón?¿Sería eficiente? *Tips*: Pide solo el planteo, no la implementación.

e) ¿Qué sucede cuando solo ejecuta con una sola instancia?

f) *Punto opcional*: El código que ejecutan las instancias esclavas, tienen un error en su lógica. ¿Cómo se podría solucionar?

### Respuestas

a) El programa está diseñado con un patrón de trabajo del estilo jefe/trabajador en el cual la única instancia maestra se encarga de generar las tareas a realizar y las delega entre las instancias esclavas.

Instancia maestra: esta inicia instanciando la cantidad total de tareas que generara y repartirá entre las estancias esclavas, la misma será igual a (nroProcesos-1)*4 tareas, el menos uno se debe a que no genera tareas para sí misma, solo lo hace para las demás instancias. Luego genera tanto tiempo de descanso como tareas vaya a repartir entre las esclavas, estos valores son generados de forma aleatoria, pero teniendo un valor máximo y mínimo configurables. Una vez creados estos valores ejecuta la función initWork, en esta le asigna a cada esclavo por medio de un mensaje una tarea y un tiempo de descanso, luego se queda a espera que vayan terminando y a medida que van notificando esto último les asigna una nueva tarea, así hasta terminar con todas. Luego de recibir el aviso de que se finalizó la última tarea les envía un mensaje a todas las instancias esclavas avisando de que se concluyó con la actividad, y luego finaliza su ejecución.

Instancias esclavas: están a la espera de un mensaje de la instancia maestra, este contiene dos cosas, un valor y un tag que puede tener dos significados. Si el tag recibido representa que se debe trabajar, el valor que lo acompaña indicara el tiempo de descanso que le corresponde al esclavo durante la realización de la tarea asignada, en este caso, se realiza un sleep del tiempo indicado y se envía un mensaje a la instancia maestra avisando que se concluyó la tarea encargada y quedando a la espera de un nuevo mensaje de esta instancia. En el caso de que el tag represente el fin de la actividad de trabajo, el esclavo marca el fin de su labor y finaliza su ejecución.    
 
  

b) Todas las instancias ejecutan la función main(), ya que hasta este punto no se distinguen entre maestra y esclavas, recién en esta función cada proceso instancia el tipo de comunicador a utilizar y obtiene su id, atributo propio de cada uno. A partir de este punto, haciendo uso del id, podemos distinguir entre la instancia maestra (id==0) y las instancias esclavas (id!=0). La instancia maestra es la única en ejecutar la función initWork() y todas las esclavas se encargan de ejecutar la función doWork.



c) Estos mensajes enviados desde la instancia maestra hacia las instancias esclavas pueden ser de dos tipos, uno para indicar un nuevo trabajo y otro para indicar la finalización de este. Para diferenciarlos se usa un tag que está contenido en la estructura utilizada para el envió del mensaje, este puede tomar dos valores uno para cada significado. Cuando una instancia esclava recibe un mensaje nuevo verifica el valor del tag de este y acorde al mismo continua su ejecución ya sea para realizar una nueva tarea o parta finalizar su trabajo.



d) Con el objetivo de realizar la suma de vectores de la siguiente manera: 
    
    R[i] = alfa * X[i] + Y[i]

Se podría crear un código que tenga el valor alfa y tres vectores:

    alfa = num
    vector X( cantidad_N ); -> Primer vector a sumar
    vector Y( cantidad_N ); -> Segundo vector a sumar
    vector R( cantidad_N ); -> Resultado

Inicialmente estarían vacíos, para más tarde llenar los valores del vector X y el vector Y en la instancia maestra.
Finalmente se podría hacer uso de la rutina MPI_Scatter para enviar a todas las instancias una porción de los vectores X e Y, para que se encarguen de realizar los cálculos únicamente para ese fragmento de datos, para mas tarde reunir todos los resultados usando la rutina MPI_Gather.
Si bien el aumento del poder de procesamiento es notorio al agregar más PCs al trabajo, la comunicación entre estas conlleva un costo adicional, y al tratarse en este caso de la resolución de un cálculo simple no sería tan rentable realizar todo este trabajo, al menos no mientras el volumen de datos no sea desorbitante. 




e) Al ejecutar con una sola instancia el programa designa a esta única como la clase maestra, sin embargo, al no haber mas instancias las cuales hagan el papel de esclavas, al momento de crear las tareas que a futuro serian asignadas, se encuentra con que la cantidad a crear dada por la formula (nroProcesos-1)*4 es igual a cero. De esta forma el programa finaliza al no tener tareas que asignar ni a quien asignárselas.



f) Solo noto dos cosas que podrían cambiarse. Por un lado, la instancia maestra envía a las instancias esclavas un mensaje que contiene un valor que en principio representa el tiempo de descanso que le corresponde a al esclavo por la realización de una tarea. Sin embargo, cuando la tarea se concluye, este mismo número es utilizado para identificar a la misma, y dado que estos valores inicialmente se pensaron para otra cosa, los mismos se repiten en varias ocasiones, dando la ilusión de que varios esclavos realizaron la misma tarea o que incluso uno realizo la misma más de una vez. Esto podría solucionarse enviando mas de un valor a los esclavos, uno que corresponda a la tarea y otro al tiempo de descanso. Por otro lado, se realiza la verificación del flag que identifica si se trata de una nueva tarea o de la finalización del trabajo luego de imprimir un mensaje haciendo alusión al nuevo trabajo recibido, aquí simplemente se podría cambiar el orden de estas sentencias.  

###2.1.2. Armado del ambiente

Es en donde se instalar, por única vez, el módulo MPI4py de Python en el cuaderno.

In [1]:
! pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp310-cp310-linux_x86_64.whl size=3365650 sha256=909a4b6a81af6a03a36b8ee9ef47ea7a43a4f05fdcfd6a4e75e5db2d3e38c45f
  Stored in directory: /root/.cache/pip/wheels/e8/1b/b5/97ec4cfccdde26e0f3590ad6e09a5242d508dff09704ef86c1
Successfully built mpi4py


### 2.1.3. Código del programa en Lenguaje Python 

In [8]:
%%writefile Ejercicio2.py
from mpi4py import MPI
import numpy as np
import time

# --------------------------------------------
# Formulario
Max_tiempo_sleep =   8#@param {type: "slider", min: 1, max: 10}
Min_tiempo_sleep =   2#@param {type: "slider", min: 0, max: 10}
# --------------------------------------------

# --------------------------------------------
# Constantes de comunicacion
WORK_FLAG = 1
END_WORK_FLAG = 2
# --------------------------------------------

def main():
    comm = MPI.COMM_WORLD # Instanciamos el tipo de comunicador a utilizar.
    id = comm.Get_rank()  # Obtenemos el id como atributo del proceso que se ejecuta.

    # Utilizamos el 0 para definir al procesos Maestro, cualquier otro id sera un esclavo.
    if (id == 0) :
        init()                                # Llamamos funcion init para eventos que requeriremos inicialmente solo 1 vez.
        numProcesses = comm.Get_size()        # Obtenemos el numero de procesos totales ejecutados.
        numTasks = (numProcesses-1)*4         # Se setea el numero de tareas.
        workTimes = generateTasks(numTasks)   # Se generan las tareas, en este caso seran 
        print("El jefe crea {} horas de descanso de sus empleados:".format(workTimes.size), flush=True)
        print(workTimes, flush=True)
        initWork(comm, workTimes, numProcesses)
    else:
        doWork(comm)

def generateTasks(numTasks):
    #TODO: Cambiar la semilla del random para que se generen efectivamente diferentes numeros.
    np.random.seed(1000) 
    return np.random.randint(low=Min_tiempo_sleep, high=Max_tiempo_sleep, size=numTasks)

def init():
  print()
  print("Version MPI4py utilizada: {}".format(MPI.Get_version()), flush=True)
  print()
  print( "------------------------------------", flush=True)
  print( "Sistema de trabajo Suizo:", flush=True)
  print( "------------------------------------", flush=True)
  print()

def initWork(comm, workTimes, numProcesses):
    totalWork = workTimes.size
    workcount = 0
    recvcount = 0

    print("Jefe enviando las tareas iniciales:", flush=True)
    for id in range(1, numProcesses):
        if workcount < totalWork:
            work=workTimes[workcount]
            comm.send(work, dest=id, tag=WORK_FLAG) # Envia mensaje de iniciar trabajo con el dato correspondiente del array.
            workcount += 1
            print("Jefe envia trabajo y {} hs de descanso al empleado {}.".format(work, id), flush=True)
    print( "------------------------------------", flush=True)

    # Mientras haya trabajo, se recibe el resultado de los empleados y se sigue enviando MAS trabajo.
    while (workcount < totalWork) :
        stat = MPI.Status() 
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat) # Recivimos resultados de los empleados.
        recvcount += 1
        workerId = stat.Get_source() # Obtenemos el identificador del empleado.
        print("Jefe recibe trabajo completado {} del empleado {}.".format(workTime, workerId), flush=True)
        #send next work
        work=workTimes[workcount]
        comm.send(work, dest=workerId, tag=WORK_FLAG) 
        workcount += 1
        print("Jefe envia nuevo trabajo y {} hs de descanso al empleado {}.".format(work, workerId), flush=True)

    while (recvcount < totalWork):
        stat = MPI.Status()
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat)
        recvcount += 1
        workerId = stat.Get_source()
        print("Jefe recibe trabajo completado {} del empleado {}.".format(workTime, workerId), flush=True)

    for id in range(1, numProcesses):
        comm.send(0, dest=id, tag=END_WORK_FLAG)


def doWork(comm):
    while(True):
        stat = MPI.Status() # Obtiene el estado actual del empleado.
        waitTime = comm.recv(source=0, tag=MPI.ANY_TAG, status=stat) # Obtiene lo enviado por el Jefe.
        print("Soy el empleado con id {}, toca descanzo por {} hs.".format(comm.Get_rank(), waitTime), flush=True)

        if (stat.Get_tag() == END_WORK_FLAG):
            print("Marca tarjeta el empleado {}.".format(comm.Get_rank()), flush=True)
            return
        time.sleep(waitTime)
        comm.send(waitTime, dest=0)

main()


Overwriting Ejercicio2.py


### 2.1.4 Ejecución del programa

In [9]:
# --------------------------------------------
#@title Parámetros de ejecución { vertical-output: true }
NRO =   5#@param {type: "number"}
# --------------------------------------------

! mpirun --allow-run-as-root --oversubscribe -np $NRO python Ejercicio2.py


Version MPI4py utilizada: (3, 1)

------------------------------------
Sistema de trabajo Suizo:
------------------------------------

El jefe crea 16 horas de descanso de sus empleados:
[5 2 5 6 3 7 2 3 7 2 3 6 5 6 6 7]
Jefe enviando las tareas iniciales:
Jefe envia trabajo y 5 hs de descanso al empleado 1.
Jefe envia trabajo y 2 hs de descanso al empleado 2.
Soy el empleado con id 2, toca descanzo por 2 hs.
Soy el empleado con id 1, toca descanzo por 5 hs.
Jefe envia trabajo y 5 hs de descanso al empleado 3.
Jefe envia trabajo y 6 hs de descanso al empleado 4.
------------------------------------
Soy el empleado con id 3, toca descanzo por 5 hs.
Soy el empleado con id 4, toca descanzo por 6 hs.
Soy el empleado con id 2, toca descanzo por 3 hs.
Jefe recibe trabajo completado 2 del empleado 2.
Jefe envia nuevo trabajo y 3 hs de descanso al empleado 2.
Jefe recibe trabajo completado 5 del empleado 1.
Jefe envia nuevo trabajo y 7 hs de descanso al empleado 1.
Jefe recibe trabajo complet

---
# Bibliografía

[1] MPI4py: https://mpi4py.readthedocs.io/en/stable/

[2] La versión oficial de MPI (Versión 4): https://www.mpi-forum.org/docs/mpi-4.0/mpi40-report.pdf

